In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import *
import matplotlib.pyplot as plt
import pyrosm
from pyrosm.data import sources
from shapely import wkt
import seaborn as sns
import glob

/home/jsepulveda/miniconda3/envs/myenv/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
recreacion_pois = pd.read_csv('/home/jsepulveda/files/recreacion_pois.csv')
recreacion_pois['geometry'] = recreacion_pois['geometry'].apply(wkt.loads)
recreacion_gdf = gpd.GeoDataFrame(recreacion_pois, crs='epsg:4326')
coord_list_r = [[x,y] for x,y in zip(recreacion_gdf['geometry'].x, recreacion_gdf['geometry'].y)]

In [3]:
educacion_pois = pd.read_csv('/home/jsepulveda/files/educacion_pois.csv')
educacion_pois['geometry'] = educacion_pois['geometry'].apply(wkt.loads)
educacion_gdf = gpd.GeoDataFrame(educacion_pois, crs='epsg:4326')
coord_list_e = [[x,y] for x,y in zip(educacion_gdf['geometry'].x, educacion_gdf['geometry'].y)]

In [4]:
from sklearn.neighbors import KernelDensity
import numpy as np

In [5]:
h = 0.5
kde_edu = KernelDensity(bandwidth=h, metric='haversine', kernel='gaussian', algorithm='ball_tree').fit(np.array(coord_list_e))
kde_recre = KernelDensity(bandwidth=h, metric='haversine', kernel='gaussian', algorithm='ball_tree').fit(np.array(coord_list_r))

In [31]:
punta_mañana = pd.read_csv('/home/jsepulveda/files/conexiones_punta_mañana.csv')
punta_mañana['geometry'] = punta_mañana['geometry'].apply(wkt.loads)
mañana_gdf = gpd.GeoDataFrame(punta_mañana, crs='epsg:4326').drop(['index','fecha','hora','dur','numb','rin','rinhead','rintail','tipo','NOM_COMUNA'],axis=1)
punta_tarde = pd.read_csv('/home/jsepulveda/files/conexiones_punta_tarde.csv')
punta_tarde['geometry'] = punta_tarde['geometry'].apply(wkt.loads)
tarde_gdf = gpd.GeoDataFrame(punta_tarde, crs='epsg:4326')

In [40]:
mañana_gdf

,numa,geometry
0,a63e7a0c3fac8a6fbf5f9f85923d7b2343f451dc12382c...,POINT (-70.58198 -33.45606)
1,2bccfac9d5402fe23ddf6b6aaf87a78b02f723e820b78f...,POINT (-70.66274 -33.49460)
2,2bccfac9d5402fe23ddf6b6aaf87a78b02f723e820b78f...,POINT (-70.64104 -33.42244)
3,2bccfac9d5402fe23ddf6b6aaf87a78b02f723e820b78f...,POINT (-70.66895 -33.41188)
4,2bccfac9d5402fe23ddf6b6aaf87a78b02f723e820b78f...,POINT (-70.68701 -33.48443)
...,...,...
1190876,c7b6e8635b91b03f1835fbfc5e3c0c172ed9dfaebe6131...,POINT (-70.77056 -33.41872)
1190877,c7b6e8635b91b03f1835fbfc5e3c0c172ed9dfaebe6131...,POINT (-70.75169 -33.49137)
1190878,9919c2fc7114df558f8e559696ea505c757ea6a2cb3b50...,POINT (-70.60406 -33.44337)
1190879,9919c2fc7114df558f8e559696ea505c757ea6a2cb3b50...,POINT (-70.60406 -33.44337)


In [35]:
i = 0

promedio_por_usuario_e = pd.DataFrame(columns=['usuario', 'rho_e', 'rho_r'])
for index, row in mañana_gdf.iterrows():
        usuario_mask = mañana_gdf['numa'] == row['numa']
        print(row['numa'])
        filtered = mañana_gdf[usuario_mask]
        coord_list = [[x,y] for x,y in zip(filtered['geometry'].x, filtered['geometry'].y)]
        promedio_u_edu = np.mean(np.exp(kde_edu.score_samples(np.array(coord_list))))
        promedio_u_recre = np.mean(np.exp(kde_recre.score_samples(np.array(coord_list))))
        promedio_por_usuario_e.loc[len(promedio_por_usuario_e)]=[row['numa'],promedio_u_edu, promedio_u_recre]
        i = i + 1
        if i > 10:
            break
    
    
promedio_por_usuario_e

a63e7a0c3fac8a6fbf5f9f85923d7b2343f451dc12382c7679582a99b6e94321
2bccfac9d5402fe23ddf6b6aaf87a78b02f723e820b78f37dad00e6e2817512b
2bccfac9d5402fe23ddf6b6aaf87a78b02f723e820b78f37dad00e6e2817512b
2bccfac9d5402fe23ddf6b6aaf87a78b02f723e820b78f37dad00e6e2817512b
2bccfac9d5402fe23ddf6b6aaf87a78b02f723e820b78f37dad00e6e2817512b
2bccfac9d5402fe23ddf6b6aaf87a78b02f723e820b78f37dad00e6e2817512b
dff071331275d8cfc2a0b1691e5632c511866b3599f14f8c41aebdad309ea8c7
dff071331275d8cfc2a0b1691e5632c511866b3599f14f8c41aebdad309ea8c7
dff071331275d8cfc2a0b1691e5632c511866b3599f14f8c41aebdad309ea8c7
53a1e050ac115c0756fbeebce8764c7911f87f1ef034f3dccf9d4cc6880d047a
53a1e050ac115c0756fbeebce8764c7911f87f1ef034f3dccf9d4cc6880d047a


,usuario,rho_e,rho_r
0,a63e7a0c3fac8a6fbf5f9f85923d7b2343f451dc12382c...,0.625351,0.629504
1,2bccfac9d5402fe23ddf6b6aaf87a78b02f723e820b78f...,0.629859,0.630124
2,2bccfac9d5402fe23ddf6b6aaf87a78b02f723e820b78f...,0.629859,0.630124
3,2bccfac9d5402fe23ddf6b6aaf87a78b02f723e820b78f...,0.629859,0.630124
4,2bccfac9d5402fe23ddf6b6aaf87a78b02f723e820b78f...,0.629859,0.630124
5,2bccfac9d5402fe23ddf6b6aaf87a78b02f723e820b78f...,0.629859,0.630124
6,dff071331275d8cfc2a0b1691e5632c511866b3599f14f...,0.630125,0.632417
7,dff071331275d8cfc2a0b1691e5632c511866b3599f14f...,0.630125,0.632417
8,dff071331275d8cfc2a0b1691e5632c511866b3599f14f...,0.630125,0.632417
9,53a1e050ac115c0756fbeebce8764c7911f87f1ef034f3...,0.627016,0.625759


In [41]:
promedio_por_usuario_e.drop_duplicates()

,usuario,rho_e,rho_r
0,a63e7a0c3fac8a6fbf5f9f85923d7b2343f451dc12382c...,0.625351,0.629504
1,2bccfac9d5402fe23ddf6b6aaf87a78b02f723e820b78f...,0.629859,0.630124
6,dff071331275d8cfc2a0b1691e5632c511866b3599f14f...,0.630125,0.632417
9,53a1e050ac115c0756fbeebce8764c7911f87f1ef034f3...,0.627016,0.625759
